## In this notebook, we will preprocess data based on data observation.

#### Data Info :
1. we have total of 20 types of documents(Text files) and total 18828 documents(text files).
2. Now our problem is to classify all the documents into any one of the class.


##### sample document
* Subject: A word of advice
* From: jcopelan@nyx.cs.du.edu (The One and Only)
* In article < 65882@mimsy.umd.edu > mangoe@cs.umd.edu (Charley Wingate) writes:

* I've said 100 times that there is no "alternative" that should think you
might have caught on by now. And there is no "alternative", but the point
is, "rationality" isn't an alternative either. The problems of metaphysical
and religious knowledge are unsolvable-- or I should say, humans cannot
solve them.
* How does that saying go: Those who say it can't be done shouldn't interrupt
those who are doing it.
Jim
--
Have you washed your brain today?

##### Email-id Preprocess

* Find all emails in the document and then get the text after the "@". and then split those texts by '.', because ww observe that before "@" is mostly organization name or person name.

* After that remove the words whose length is less than or equal to 2 and also remove 'com' (occured in all emails) word and then combine those words by space. 

* In one doc, if we have 2 or more mails, get all, Eg:[test@dm1.d.com, test2@dm2.dm3.com]-->[dm1.d.com, dm3.dm4.com]-->[dm1,d,com,dm2,dm3,com]-> [dm1,dm2,dm3]-->"dm1 dm2 dm3" append all those into one list/array. ( This will give length of 18828 sentences i.e one list for each of the document).

* In the above sample document there are emails [jcopelan@nyx.cs.du.edu,
65882@mimsy.umd.edu, mangoe@cs.umd.edu]

preprocessing: [jcopelan@nyx.cs.du.edu, 65882@mimsy.umd.edu, mangoe@cs.umd.edu] ==> [nyx cs du edu mimsy umd edu cs umd edu] ==> [nyx edu mimsy umd edu umd edu]

*  Replace all the emails by space in the original text.


##### Subject text Preprocess

* Get subject of the text i.e. get the total lines where "Subject:" occur and remove.

* those words which are before the ":" remove the newlines, tabs, punctuations, any special chars.

* Eg: if we have sentance like "Subject: Re: Gospel Dating @ \r\r\n" --> You have to get "Gospel Dating" Save all this data into another list/array.

* After you store it in the list, Replace those sentances in original text by space.

##### Text Preprocess Steps

* Delete all the sentances where sentence starts with "Write to:" or "From:".

* Delete all the tags like "< anyword >"

* Delete all the data which are present in the brackets.
In many text data, we observed that, they maintained the explanation of sentence
or translation of sentence to another language in brackets so remove all those.

* Remove all the newlines('\n'), tabs('\t'), "-", "\". Remove all the words which ends with ":". Eg: "Anyword:"

* In the above sample document check the 4nd line, we should remove that "writes:"

* Decontractions, replace words like below to full words. please check the donors choose preprocessing for this Eg: can't -> can not, 's -> is, i've -> i have, i'm -> i am, you're -> you are, i'll --> i will There is no order to do point 6 to 10. but you have to get final output correctly

* Do chunking on the text you have after above preprocessing. Text chunking, also referred to as shallow parsing, is a task that follows Part-Of-Speech
Tagging and that adds more structure to the sentence. So it combines the some phrases, named entities into single word. So after that combine all those phrases/named entities by separating "_". And remove the phrases/named entities if that is a "Person". You can use nltk.ne_chunk to get these.
Below we have given one example. please go through it.

* Replace all the digits with space i.e delete all the digits.

* After doing above points, we observed there might be few word's like
"_word_" (i.e starting and ending with the _), "_word" (i.e starting with the _,"word_" (i.e ending with the _) remove the _ from these type of words.

* We also observed some words like "OneLetter_word"- eg: d_berlin,
"TwoLetters_word" - eg: dr_berlin , in these words we remove the "OneLetter_"(d_berlin ==> berlin) and "TwoLetters_" (de_berlin ==> berlin). i.e remove the words which are length less than or equal to 2 after spliiting those words by "_".

* Convert all the words into lower case and lowe case and remove the words which are greater than or equal to 15 or less than or equal to 2.

* replace all the words except "A-Za-z_" with space.

* Now You got Preprocessed Text, email, subject. create a dataframe with those.
Below are the columns of the df.

In [ ]:
''' Import packages '''

import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import re
import os
from email.utils import parseaddr
from tqdm.notebook import tqdm
import nltk
from nltk import word_tokenize, pos_tag, ne_chunk
from nltk.chunk import tree2conlltags
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.layers import Dense
from keras.layers.core import Activation
from keras.layers import Dense, Input, Flatten, Dropout
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Sequential
from tensorflow.keras import regularizers
from keras.layers import concatenate
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from tensorflow.keras import backend as K
import datetime
from tensorflow.keras.utils import to_categorical
import tensorflow.keras.layers as layer

In [ ]:
''' Download few nltk package '''

nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('punkt')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# download globe model tokens https://nlp.stanford.edu/projects/glove/
!curl --header "Host: downloads.cs.stanford.edu" --header "User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36" --header "Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9" --header "Accept-Language: en-US,en;q=0.9" --header "Referer: https://nlp.stanford.edu/" --header "Cookie: _ga=GA1.2.545929601.1671976005; _gid=GA1.2.2065625101.1671976005; _gat=1" --header "Connection: keep-alive" "https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip" -L -o "glove.6B.zip"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  822M  100  822M    0     0  4485k      0  0:03:07  0:03:07 --:--:-- 3423k


In [ ]:
!unzip "/content/glove.6B.zip" -d "/content/glove"

Archive:  /content/glove.6B.zip
  inflating: /content/glove/glove.6B.50d.txt  
  inflating: /content/glove/glove.6B.100d.txt  
  inflating: /content/glove/glove.6B.200d.txt  
  inflating: /content/glove/glove.6B.300d.txt  


In [ ]:
!unzip  "/content/drive/MyDrive/documents.zip" -d "/content"

#### Data Preprocess

In [ ]:
# This code reference from this link - https://stackoverflow.com/a/31837224/4084039

def split_loc(sentence):

    '''
    Inputs :
        name    : sentence
        type    : str  
    
    Ouput : 
        name    : sentence
        type    : str
        content : modification on input

    Process :
        This function take input as string and using pos-tag 
        check if word is city name then add "_" between two words 
        and return new modified string. 
    '''

    # convert string into tokens
    word = nltk.word_tokenize(sentence)

    # extract pos tag from strint
    pos_tag = nltk.pos_tag(word)

    chunk = nltk.ne_chunk(pos_tag)
    
    old = [ " ".join(w for w, t in ele) for ele in chunk if isinstance(ele, nltk.Tree)]

    # replace underscore
    new = [w.replace(w, "_".join(w.split())) for w in old]

    d = dict(zip(old, new))

    # replace new token to old in string
    for i, j in d.items():
        sentence = sentence.replace(i, j)
    return sentence

In [ ]:
def process(Input_text):

    '''
    Inputs :
        name    : Input_text
        type    : str
        content : text data (text file) 
    
    Ouput : 
        name    : email_file 
        type    : list
        content : list of strings

        name    : subject_file 
        type    : list
        content : list of strings

        name    : text_file 
        type    : list
        content : list of strings  

    Process :
        This function take input as python text file and using different methods
        extract email, email subject and email text and preprocess it.
    '''
    

    email_file = " "
    subject_file = ""
    text_file = [ ]
    for line in Input_text:

    ###### Email id preprocess ######

        # extract email ids
        email1 = re.findall(r'[\w\.-]+@[\w\.-]+', line)
        e = ''
        for i in email1:
            remove_email = re.sub(i, "", line)
            email = i.split('@')[1]
            email = " ".join(email.split('.'))
            email = re.sub('com|COM', "", email)
            email = email if len(email) > 2 else ""
            e += " " + email
        
        email_file += e

    ###### Subject Text preprocess #####

        subject1 = line if 'Subject:' in line else ""
        subject = re.sub("\n", "", subject1)
        subject = re.split('re:|Re:|Subject:|:', subject)
        subject = "".join(subject[1:])
        subject = re.sub('[^A-Za-z_]', ' ', subject)
        subject = subject.strip(" ")
        subject_file += subject
        line = line.replace(subject1,"")

    ###### Email text preprocess ######

        line = line if 'From:' not in line else ""
        line = line if 'Write to:' not in line else ""

        #Decontractions
        line = re.sub(r"won't", "will not", line)
        line = re.sub(r"can\'t", "can not", line)
        line = re.sub(r"n\'t", " not", line)
        line = re.sub(r"\'re", " are", line)
        line = re.sub(r"\'s", " is", line)
        line = re.sub(r"\'d", " would", line)
        line = re.sub(r"\'ll", " will", line)
        line = re.sub(r"\'t", " not", line)
        line = re.sub(r"\'ve", " have", line)
        line = re.sub(r"\'m", " am", line)

        #remove the word with end of ":"
        line = "".join(line.split(':')[1:]) if ':' in line else line
        #remove the brackets and its context
        line = re.sub("([\(\[]).*?([\)\]])", "", line)
        #remove the tags
        line = re.sub('<.*?>', "", line)
        #remove the special characters
        line = re.sub('\W+', " ", line)
        #remove extra space
        line = " ".join(line.split())
        #remove the special characters
        line = re.sub('\W+', " ", line)
        #remove the special char
        line = re.sub('[\W\_]',' ',line)
        #find the grp place and split by '_'
        line = split_loc(line)
        #convert into the lowercase
        line = line.lower()
        #convert below form except ""
        # line = re.sub('^[A-Za-z_]', '', line)
        #remove the digits
        remove_digits = str.maketrans('', '', digits)
        line = line.translate(remove_digits)
        #remove len(string) less than or equal to in string
        line = " ".join([i for i in line.split() if len(i) > 2 ])
        text_file.append(line)

    text_file = " ".join(' '.join(text_file).split())

    return (email_file.lower(), subject_file.lower(), text_file.lower())


In [ ]:
text_dir = os.listdir("/content/documents")

In [ ]:
''' pre process text subject and email '''

pre_process_text = [ ]
pre_process_email = [ ]
pre_process_subject = [ ]

for text in tqdm(text_dir):
    fileName = '/content/documents/' + str(text)
    with open(fileName, 'r', encoding=' iso-8859-1') as file:
        lines = file.readlines()
        email, subject, text = process(lines)
        pre_process_email.append(email)
        pre_process_subject.append(subject)
        pre_process_text.append(text)

In [ ]:
text_df = pd.DataFrame()
text_df['text'] = pre_process_text
text_df['email_id'] = pre_process_email
text_df['subject'] = pre_process_subject

In [ ]:
''' pre process raw text and labels '''

raw_text = [ ]
class_label = [ ]

for text in tqdm(text_dir):

    m = re.search('_', text)
    label = text[: m.end()-1]
    class_label.append(label)
    fileName = '/content/documents/' + str(text)

    with open(fileName, 'r', encoding=' iso-8859-1') as file:
        
        lines = file.readlines()
        lines = " ".join(' '.join(lines).split())
        raw_text.append(lines)

In [ ]:
text_df['raw_text'] = raw_text
text_df['class_label'] = class_label

In [ ]:
text_df

,text,email_id,subject,raw_text,class_label
0,his was posted the firearms politics mailing l...,hound dazixca ingr hound dazixca ingr,randy weaver trail - day 3,From: crphilli@hound.dazixca.ingr.com (Ron Phi...,talk.politics.guns
1,having problem with truetype fonts windows hav...,gandalf fl bs dlr de gandalf fl bs dlr de,truetype font mix-up times=>cyrillic,From: FL2G@gandalf.fl.bs.dlr.de (Reiner Suikat...,comp.os.ms-windows.misc
2,does the radio electronics free information ca...,panix acsu buffalo edu ubvmsb cc buffalo ed...,radio electronics free information card,From: schuster@panix.com (Michael Schuster) Su...,sci.electronics
3,uppose have boolean function which minimal sum...,carson u washington edu ringer cs utsa edu r...,minimal boolean circuit,From: whit@carson.u.washington.edu (John Whitm...,sci.electronics
4,ello name john and have the following comic bo...,iscsvax uni edu iscsvax uni edu iscsvax uni edu,****comic book sale****,From: oeth6050@iscsvax.uni.edu Subject: ****CO...,misc.forsale
...,...,...,...,...,...
18823,ate begets more hate never love consider some ...,world std athos rutgers edu prism gatech edu,hate the sin...,From: pduggan@world.std.com (Paul C Duggan) Su...,soc.religion.christian
18824,article arromdee jyusenkyou jhu edu think many...,newton apple blaze cs jhu edu jyusenkyou cs...,yet more rushdie [ islamic law],From: sandvik@newton.apple.com (Kent Sandvik) ...,alt.atheism
18825,ould some one tell what har lap err the chip c...,spartan ac brocku ca nmt edu jupiter nmt edu...,help! phar lap???,From: tmc@spartan.ac.BrockU.CA (Tim Ciceran) S...,comp.sys.ibm.pc.hardware
18826,abital planets are also dependent what kind pl...,ucsu colorado edu aurora alaska edu aurora a...,human habitale planets?,From: fcrary@ucsu.Colorado.EDU (Frank Crary) S...,sci.space


In [ ]:
# remove special char from string
text_df['subject'].replace({'[^A-Za-z_]' : ' '}, regex=True, inplace=True)

In [ ]:
text_df['combine'] = text_df['text'] + text_df[ 'subject'] + text_df['email_id']

In [ ]:
text_df

,text,email_id,subject,raw_text,class_label,combine
0,his was posted the firearms politics mailing l...,hound dazixca ingr hound dazixca ingr,randy weaver trail day,From: crphilli@hound.dazixca.ingr.com (Ron Phi...,talk.politics.guns,his was posted the firearms politics mailing l...
1,having problem with truetype fonts windows hav...,gandalf fl bs dlr de gandalf fl bs dlr de,truetype font mix up times cyrillic,From: FL2G@gandalf.fl.bs.dlr.de (Reiner Suikat...,comp.os.ms-windows.misc,having problem with truetype fonts windows hav...
2,does the radio electronics free information ca...,panix acsu buffalo edu ubvmsb cc buffalo ed...,radio electronics free information card,From: schuster@panix.com (Michael Schuster) Su...,sci.electronics,does the radio electronics free information ca...
3,uppose have boolean function which minimal sum...,carson u washington edu ringer cs utsa edu r...,minimal boolean circuit,From: whit@carson.u.washington.edu (John Whitm...,sci.electronics,uppose have boolean function which minimal sum...
4,ello name john and have the following comic bo...,iscsvax uni edu iscsvax uni edu iscsvax uni edu,comic book sale,From: oeth6050@iscsvax.uni.edu Subject: ****CO...,misc.forsale,ello name john and have the following comic bo...
...,...,...,...,...,...,...
18823,ate begets more hate never love consider some ...,world std athos rutgers edu prism gatech edu,hate the sin,From: pduggan@world.std.com (Paul C Duggan) Su...,soc.religion.christian,ate begets more hate never love consider some ...
18824,article arromdee jyusenkyou jhu edu think many...,newton apple blaze cs jhu edu jyusenkyou cs...,yet more rushdie islamic law,From: sandvik@newton.apple.com (Kent Sandvik) ...,alt.atheism,article arromdee jyusenkyou jhu edu think many...
18825,ould some one tell what har lap err the chip c...,spartan ac brocku ca nmt edu jupiter nmt edu...,help phar lap,From: tmc@spartan.ac.BrockU.CA (Tim Ciceran) S...,comp.sys.ibm.pc.hardware,ould some one tell what har lap err the chip c...
18826,abital planets are also dependent what kind pl...,ucsu colorado edu aurora alaska edu aurora a...,human habitale planets,From: fcrary@ucsu.Colorado.EDU (Frank Crary) S...,sci.space,abital planets are also dependent what kind pl...


In [ ]:
# check nan values in dataset
text_df.dropna(inplace=True)

In [ ]:
text_df.shape

(18789, 6)

In [ ]:
text_df.to_csv("/content/drive/MyDrive/email_data_preprocess2.csv", index=False)